# テキストの分類
テキスト分類は、以下の順に行います。

1. 前処理
2. 数値表現化 (CountVectorizer, TfIdfVectorizer)
3. 分類器を学習

本ノートブックでは、Wikipedia のエントリ分類を通じて、テキスト分類器を作成します。

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [0]:
import joblib
import MeCab
import numpy as np
import pandas as pd
import re

from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC

## データの読み込み

In [0]:
# タブ (\t) 区切りファイルを読み込む
df = pd.read_csv("./drive/My Drive/0_インテグ作業/data/EPG_checking (1).csv")

In [0]:
df

In [0]:
mecab.parse(df["sharp_detail_epg"][2])





In [0]:
# df.iloc[1]["sharp_detail_epg_tknz"] = "hoge"
# df.iloc[1]["sharp_detail_epg_tknz"] = pd.Series(["hoge"])

df["sharp_detail_epg_tknz"] = np.nan

In [0]:
df

In [0]:
# mecab = MeCab.Tagger()

# # text_tokenized = []
# for i, text in enumerate( df['sharp_detail_epg']):
#   try:
#     tmp_node = mecab.parseToNode(text)
#     while tmp_node:
#       print(tmp_node.feature+" ")

#       tmp_node.next


#     # df["sharp_detail_epg_tknz"].iloc[i] = tmp_node.surface
#     # break
#   except:
#     print("p")
#     pass
#     # print(text)


In [0]:
# df["sharp_text_epg_tknz"]
df.columns

Index(['Unnamed: 0', 'drama_key', 'drama_title', 'sharp_title',
       'sharp_num_cnt', 'sharp_num_epg', 'start_time', 'sharp_detail_epg'],
      dtype='object')

In [0]:
df.iloc[1]

Unnamed: 0                                                          1
drama_key                                                 1910_CX_月21
drama_title                                                    シャーロック
sharp_title                 番組名シャーロック【探偵×医師最強バディ始動!天才VS聖女の謎解き心理戦】　#02
sharp_num_cnt                                                       2
sharp_num_epg                                                 ['#02']
start_time                                               ['21', '22']
sharp_detail_epg    番組詳細「FIVBワールドカップバレーボール2019　男子　日本×ブラジル」延長の際、放送時...
Name: 1, dtype: object

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:


X = vectorizer.fit_transform(
    [str(i) for i in df["sharp_detail_epg_tknz"].values]
)

KeyError: ignored

In [0]:
X

In [0]:
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

In [0]:
vectorizer.idf_

## 形態素解析
---

MeCab を利用した形態素解析と、簡単な処理の例を示します。

### -O wakati オプション無しでの解析
品詞を使うなど、高度な解析が必要な場合は -O wakati オプションを外し、`parseToNode` 等を用いて解析します (文字列処理に慣れたら、結果を一括で出力する `parse` を使い、文字列処理により結果を取り出すこともできます)。

In [0]:
mecab = MeCab.Tagger()

# バグ回避用
mecab.parse("")

'EOS\n'

In [0]:
# node が None になるまで、node.next を辿り続ける
node = mecab.parseToNode("本日も晴天なり")

while node:
    # 単語は surface、品詞は feature に格納されている
    print(node.surface + "\t" + node.feature)
    node = node.next

	BOS/EOS,*,*,*,*,*,*,*,*
本日	名詞,副詞可能,*,*,*,*,本日,ホンジツ,ホンジツ
も	助詞,係助詞,*,*,*,*,も,モ,モ
晴天	名詞,一般,*,*,*,*,晴天,セイテン,セイテン
なり	助動詞,*,*,*,文語・ナリ,基本形,なり,ナリ,ナリ
	BOS/EOS,*,*,*,*,*,*,*,*


↑ BOS, EOS は、それぞれ文頭・文末を意味する特別な品詞です (無視できます)。

In [0]:
# BOS/EOS, 助詞, 助動詞を除外する例
node = mecab.parseToNode(df.text.iloc[0])
while node:
    if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and not node.feature.startswith("助動詞"):
        print(node.surface + "\t" + node.feature)
    node = node.next

くし	名詞,一般,*,*,*,*,くし,クシ,クシ
型	名詞,接尾,一般,*,*,*,型,ガタ,ガタ
関数	名詞,一般,*,*,*,*,関数,カンスウ,カンスー
くし	名詞,一般,*,*,*,*,くし,クシ,クシ
型	名詞,接尾,一般,*,*,*,型,ガタ,ガタ
関数	名詞,一般,*,*,*,*,関数,カンスウ,カンスー
（	記号,括弧開,*,*,*,*,（,（,（
くし	名詞,一般,*,*,*,*,くし,クシ,クシ
たかん	動詞,自立,*,*,五段・ラ行,体言接続特殊,たかる,タカン,タカン
すう	動詞,自立,*,*,五段・ワ行促音便,基本形,すう,スウ,スウ
、	記号,読点,*,*,*,*,、,、,、
）	記号,括弧閉,*,*,*,*,）,）,）
、	記号,読点,*,*,*,*,、,、,、
デルタ	名詞,一般,*,*,*,*,デルタ,デルタ,デルタ
関数	名詞,一般,*,*,*,*,関数,カンスウ,カンスー
一定	名詞,サ変接続,*,*,*,*,一定,イッテイ,イッテイ
間隔	名詞,一般,*,*,*,*,間隔,カンカク,カンカク
並べ	動詞,自立,*,*,一段,連用形,並べる,ナラベ,ナラベ
超	接頭詞,名詞接続,*,*,*,*,超,チョウ,チョー
関数	名詞,一般,*,*,*,*,関数,カンスウ,カンスー
。	記号,句点,*,*,*,*,。,。,。
英語	名詞,一般,*,*,*,*,英語,エイゴ,エイゴ
コム	名詞,固有名詞,組織,*,*,*,コム,コム,コム
関数	名詞,一般,*,*,*,*,関数,カンスウ,カンスー
。	記号,句点,*,*,*,*,。,。,。
概	名詞,サ変接続,*,*,*,*,概,ガイ,ガイ
形	名詞,接尾,一般,*,*,*,形,ガタ,ガタ
キリル	名詞,一般,*,*,*,*,*
文字	名詞,一般,*,*,*,*,文字,モジ,モジ
「	記号,括弧開,*,*,*,*,「,「,「
Ш	名詞,固有名詞,組織,*,*,*,*
」	記号,括弧閉,*,*,*,*,」,」,」
たとえ	動詞,自立,*,*,一段,連用形,たとえる,タトエ,タトエ
シャー	名詞,固有名詞,人名,一般,*,*,シャー,シャー,シャー
関数	名詞,一般,*,*,*,*,関数,カンスウ,カンスー
（	記号,括弧開,*,*,*,*,（,（,（
しゃ	動詞,接尾,*,*,五段・サ行,

In [0]:
mecab = MeCab.Tagger("-O wakati")

text_tokenized = []
for text in df['text']:
    text_tokenized.append(mecab.parse(text))
    
df['text_tokenized'] = text_tokenized

'くし 型 関数 くし 型 関数 （ くし が たかん すう 、 ） は 、 デルタ 関数 を 一定 の 間隔 で 並べ た 超 関数 。 英語 から コム 関数 とも 。 概 形 を キリル 文字 の 「 Ш 」 に たとえ て シャー 関数 （ しゃ ー かんす う 、 ） と も 呼ば れる 。 また わかり やすく 周期 的 デルタ 関数 と も 呼ば れる 。 連続 関数 と の 積 を 取る こと により 、 一定 間隔 で 離散 化 （ サンプリング ） し た 数値 列 を 得る こと が できる 。 この ため 理想 的 な サンプラー の モデル として も 扱わ れる 。 が 成り立つ 。 \n'

In [0]:
df

,category,text,text_tokenized
0,mathematics,くし型関数 くし型関数（くしがたかんすう、）は、デルタ関数を一定の間隔で並べた超関数。英語か...,くし 型 関数 くし 型 関数 （ くし が たかん すう 、 ） は 、 デルタ 関数 を...
1,mathematics,"フィッティングの補題 数学において、の補題 (FITTING LEMMA) は、""M"" が直...",フィッティング の 補題 数学 において 、 の 補題 ( FITTING LEMMA ) ...
2,philosophy,理性 理性（りせい、→→→）とは、人間に本来的に備わっているとされる知的能力の一つである。言...,理性 理性 （ り せい 、 → → → ） と は 、 人間 に 本来 的 に 備わっ て...
3,mathematics,位相線型環 数学の函数解析学における位相線型環（いそうせんけいかん、; 位相多元環、位相代数...,位相 線型 環 数学 の 函数 解析 学 における 位相 線型 環 （ い そう せん けい...
4,mathematics,調和解析 数学の一分野としての調和解析（ちょうわかいせき、）は、関数や信号を基本波の重ね合わ...,調和 解析 数学 の 一 分野 として の 調和 解析 （ ちょう わかい せき 、 ） は...
...,...,...,...
995,mathematics,射影線型群 数学における射影線型群（しゃえいせんけいぐん、）あるいは射影一般線型群（しゃえい...,射影 線型 群 数学 における 射影 線型 群 （ し ゃえいせんけいぐん 、 ） あるいは...
996,mathematics,円柱 (数学) 数学において円柱（えんちゅう、）とは二次曲面（三次元空間内の曲面）の一種で、...,円柱 ( 数学 ) 数学 において 円柱 （ えん ち ゅう 、 ） と は 二 次 曲面 ...
997,mathematics,"エルミート作用素 エルミート作用素（エルミートさようそ、""HERMITIAN OPERATO...","エルミート 作用素 エルミート 作用素 （ エルミート さ よう そ 、 "" HERMITI..."
998,philosophy,"パスカルの賭け パスカルの賭け（パスカルのかけ、, ）は、フランスの哲学者ブレーズ・パスカル...","パスカル の 賭け パスカル の 賭け （ パスカル の かけ 、 , ） は 、 フランス..."


## 単語文書行列の作成
---
単語文書行列は、`CountVectorizer` を用いることで作成できます。

### 簡単な例

In [0]:
# token_pattern は、デフォルトだと1文字の単語を除外するので、除外しないように設定する
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:
# スペース区切りの文書をリストで与える
X = vectorizer.fit_transform([
    "This is a pen",
    "I am not a pen",
    "Hello"
])

In [0]:
X

<3x8 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

単語文書行列は、`scipy.sparse` の SparseMatrix (疎行列) の形で帰ります。疎行列形式は、単語文書行列のように疎なデータ (行列内の多くの値が 0) に対し、メモリ効率良く格納できると言う利点を持ちます。

デバッグ用途等で密行列に変換したい場合は、toarray() を呼び出します。

In [0]:
X.toarray()

array([[1, 0, 0, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0]])

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

,a,am,hello,i,is,not,pen,this
0,1,0,0,0,1,0,1,1
1,1,1,0,1,0,1,1,0
2,0,0,1,0,0,0,0,0


CountVectorizer が生成する単語文書行列では、行 (横) 方向が1つの文書を表し、それぞれの列に単語の出現回数が格納されます (※単語文書行列と言った場合、列方向が文書を表す場合もあるので、他の文献を読む際は注意が必要です)。

どの列がどの単語と対応しているかを知るためには、vectorizer の vocabulary_ を参照します。

In [0]:
vectorizer.vocabulary_

{'a': 0, 'am': 1, 'hello': 2, 'i': 3, 'is': 4, 'not': 5, 'pen': 6, 'this': 7}

新しい文書に対しては、transform で変換します。

なお、fit\_transform の際に出てこなかった単語 (= vocabulary_ にない単語) については、行列内に登場しません。

In [0]:
# transform も、リスト形式で与える必要があるので注意
X_new = vectorizer.transform([
    "Hello I like this pen"
])
X_new.toarray()

array([[0, 0, 1, 1, 0, 0, 1, 1]])

### 日本語データに適用
前処理済みの日本語データに適用してみます。DataFrame にスペース区切りのテキストを準備している場合、DataFrame の列を参照させたものをそのまま与えることができます。

In [0]:
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X

<1000x30875 sparse matrix of type '<class 'numpy.int64'>'
	with 227179 stored elements in Compressed Sparse Row format>

In [0]:
X[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

↑この規模の単語文書行列になると、行列のサイズ (文書数x単語数) と比較し、値が入っている場所をほとんど無いことが分かります。

In [0]:
vectorizer.vocabulary_

{'くし': 6144,
 '型': 18802,
 '関数': 30013,
 'が': 6073,
 'たかん': 6613,
 'すう': 6449,
 'は': 7144,
 'デルタ': 11461,
 'を': 7843,
 '一定': 14905,
 'の': 7116,
 '間隔': 29996,
 'で': 6838,
 '並べ': 15334,
 'た': 6594,
 '超': 28848,
 '英語': 27409,
 'から': 6053,
 'コム': 9836,
 'とも': 6910,
 '概': 23519,
 '形': 20825,
 'キリル': 9357,
 '文字': 22466,
 'ш': 5583,
 'に': 7056,
 'たとえ': 6650,
 'て': 6823,
 'シャー': 10264,
 'しゃ': 6402,
 'ー': 14873,
 'かんす': 6067,
 'う': 5825,
 'と': 6857,
 'も': 7506,
 '呼ば': 18361,
 'れる': 7803,
 'また': 7397,
 'わかり': 7819,
 'やすく': 7590,
 '周期': 18345,
 '的': 25447,
 '連続': 29323,
 '積': 26104,
 '取る': 17934,
 'こと': 6253,
 'により': 7084,
 '離散': 30265,
 '化': 17502,
 'サンプリング': 10106,
 'し': 6355,
 '数値': 22415,
 '列': 17088,
 '得る': 20972,
 'できる': 6844,
 'この': 6262,
 'ため': 6668,
 '理想': 25055,
 'な': 6973,
 'サンプラー': 10105,
 'モデル': 13912,
 'として': 6884,
 '扱わ': 21703,
 '成り立つ': 21501,
 'フィッティング': 12522,
 '補題': 27791,
 '数学': 22421,
 'において': 7060,
 'fitting': 2581,
 'lemma': 3436,
 'm': 3558,
 '直': 25531,
 '既': 22639,
 '約': 

## TF-IDF変換
---

TF-IDF 変換を行うと、単語の出現回数に、他の文書全体と比較した際の希少性 (レア度) で重みを付けることができます。

TF-IDF 変換済みの単語文書行列は、`TfidfVectorizer` で作成できます。

### 簡単な例

In [0]:
# token_pattern は、デフォルトだと1文字の単語を除外するので、除外しないように設定する
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:
df[""]

In [0]:
# スペース区切りの文書をリストで与える
X = vectorizer.fit_transform([
    "This is a pen",
    "I am not a pen",
    "Hello"
])
X.toarray()

array([[0.42804604, 0.        , 0.        , 0.        , 0.5628291 ,
        0.        , 0.42804604, 0.5628291 ],
       [0.37302199, 0.49047908, 0.        , 0.49047908, 0.        ,
        0.49047908, 0.37302199, 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

,a,am,hello,i,is,not,pen,this
0,0.428046,0.000000,0.0,0.000000,0.562829,0.000000,0.428046,0.562829
1,0.373022,0.490479,0.0,0.490479,0.000000,0.490479,0.373022,0.000000
2,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000


他の文書でも出現している単語については、同じ出現回数でも重みが低くなっていることが分かります。各単語の重みは、`idf_` で見ることができます。確認してみましょう。

In [0]:
vectorizer.idf_

array([1.28768207, 1.69314718, 1.69314718, 1.69314718, 1.69314718,
       1.69314718, 1.28768207, 1.69314718])

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(np.atleast_2d(vectorizer.idf_), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

,a,am,hello,i,is,not,pen,this
0,1.287682,1.693147,1.693147,1.693147,1.693147,1.693147,1.287682,1.693147


デフォルトでは、各文書のノルムが1になるように正規化されています (norm="l2")。確認してみましょう。

In [0]:
np.linalg.norm(X.toarray(), axis=1)

array([1., 1., 1.])

### 日本語データに適用
CountVectorizer の場合と同様に、前処理済みの日本語データに適用してみます。

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

IDF 値の大きい単語・小さい単語を確認してみましょう。

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.Series(vectorizer.idf_, index=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ]) \
    .to_frame("idf") \
    .sort_values("idf", ascending=False)

,idf
リゾーマタ,7.215608
下がら,7.215608
下人,7.215608
下中,7.215608
下ろす,7.215608
...,...
に,1.105250
で,1.080043
を,1.072496
は,1.014085


## テキスト分類
---
定量化したテキスト情報を使うことで、テキストをカテゴリー別に分類してみましょう

In [0]:
# テストデータ
df_test = pd.read_csv("dataset/wikipedia-test.txt", sep="\t")
df_test.head()

,category,text
0,philosophy,細見和之 細見和之（ほそみ かずゆき、1962年2月27日）は、日本の詩人、京都大学教授、大...
1,mathematics,グライバッハ標準形 計算機科学において、文脈自由言語の全ての生成規則が次のように書けるとき、...
2,mathematics,"淡中圏 淡中圏（たんなかけん、TANNAKIAN CATEGORY）とは与えられた体""K""に..."
3,mathematics,根岸世雄 根岸 世雄（ねぎし ときお、1929年 - 2005年1月26日）は、日本の数学者...
4,mathematics,CEF CEF


### 単語文書行列 (TF-IDF 適用済み) の作成
単語文章行列を作成する前に、まずは文章を形態素解析します。

In [0]:
mecab = MeCab.Tagger("-O wakati")

text_tokenized = []
for text in df_test['text']:
    text_tokenized.append(mecab.parse(text))
    
df_test['text_tokenized'] = text_tokenized

続いて、単語文章行列を学習データテストデータでそれぞれ生成します。<br>
この時、2つの単語文章行列は同じ次元数でなければならないので、`transform`を使用します

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X_test = vectorizer.transform(df_test["text_tokenized"])

### 分類器の適用・比較
単語文書行列ができたら、後はこれまでやってきた機械学習と同様です。

いくつかの分類器で、性能を比較してみましょう。

In [0]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_true, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

### ロジスティック回帰

In [0]:
clf_lr = LogisticRegression(n_jobs=-1)
clf_lr.fit(X, df["category"])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

推定結果は predict で得ることができます (他の分類器でも同様)。

In [0]:
clf_lr.predict(X_test)

array(['philosophy', 'mathematics', 'mathematics', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'philosophy',
       'philosophy', 'philosophy', 'philosophy', 'philosophy',
       'mathematics', 'mathematics', 'mathematics', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'philosophy',
       'mathematics', 'mathematics', 'philosophy', 'mathematics',
       'mathematics', 'philosophy', 'philosophy', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'mathematics',
       'mathematics', 'mathematics', 'mathematics', 'philosophy',
       'philosophy', 'mathematics', 'philosophy', 'mathematics',
       'mathematics', 'mathematics', 'mathematics', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'mathematics',
       'philosophy', 'philosophy', 'mathematics', 'philosophy',
       'mathematics', 'mathematics', 'mathematics', 'mathematics',
       'mathemat

In [0]:
y_test_pred = clf_lr.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

Accuracy:
0.89

Report:
              precision    recall  f1-score   support

 mathematics       0.89      0.96      0.92        67
  philosophy       0.89      0.76      0.82        33

    accuracy                           0.89       100
   macro avg       0.89      0.86      0.87       100
weighted avg       0.89      0.89      0.89       100

Confusion matrix:
[[64  3]
 [ 8 25]]


### SVM (線形カーネル)

In [0]:
clf_svc = LinearSVC()
clf_svc.fit(X, df["category"])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
y_test_pred = clf_svc.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

Accuracy:
0.92

Report:
              precision    recall  f1-score   support

 mathematics       0.93      0.96      0.94        67
  philosophy       0.90      0.85      0.88        33

    accuracy                           0.92       100
   macro avg       0.92      0.90      0.91       100
weighted avg       0.92      0.92      0.92       100

Confusion matrix:
[[64  3]
 [ 5 28]]


### Random Forest

In [0]:
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_rf.fit(X, df["category"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [0]:
y_test_pred = clf_rf.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

Accuracy:
0.89

Report:
              precision    recall  f1-score   support

 mathematics       0.89      0.96      0.92        67
  philosophy       0.89      0.76      0.82        33

    accuracy                           0.89       100
   macro avg       0.89      0.86      0.87       100
weighted avg       0.89      0.89      0.89       100

Confusion matrix:
[[64  3]
 [ 8 25]]


## 【演習1】
上記いずれかのアルゴリズムのパラメータを変更して、分類精度を高めてみましょう。

In [0]:
# Enter your code here

### モデルの保存
`pickle` でも保存できますが、`joblib` を使い、ファイル名末尾に `.gz`, `.bz2` 等を指定すると、自動的に圧縮してくれます。読み込み時も同様、自動的に解凍して読み込んでくれます。

言語処理ではモデルが大容量になることが多いため、モデルの圧縮は重要です。

In [0]:
joblib.dump(clf_rf, "wikipedia_category_classifier.pkl.gz")

['wikipedia_category_classifier.pkl.gz']

### モデル読み込みテスト

In [0]:
clf_rf_restored = joblib.load("wikipedia_category_classifier.pkl.gz")
clf_rf_restored

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

読み込んだモデルは、上記ですでに学習済みなので、`predict`で予測が行えます

In [0]:
clf_rf_restored.predict(X_test)

array(['philosophy', 'mathematics', 'mathematics', 'mathematics',
       'mathematics', 'mathematics', 'mathematics', 'philosophy',
       'philosophy', 'mathematics', 'philosophy', 'philosophy',
       'mathematics', 'mathematics', 'mathematics', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'philosophy',
       'mathematics', 'philosophy', 'philosophy', 'mathematics',
       'mathematics', 'mathematics', 'philosophy', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'mathematics',
       'mathematics', 'mathematics', 'mathematics', 'philosophy',
       'philosophy', 'mathematics', 'philosophy', 'mathematics',
       'mathematics', 'mathematics', 'mathematics', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'mathematics',
       'mathematics', 'philosophy', 'mathematics', 'philosophy',
       'philosophy', 'philosophy', 'mathematics', 'philosophy',
       'mathematics', 'mathematics', 'mathematics', 'mathematics',
       'mathema

## 【演習2】
演習1で構築したモデルをjoblibを使って保存してみましょう。

In [0]:
# Enter your code here